#**Inteligencia de negocios - ISIS3301**

#**Proyecto 1 - Analitica de texto**

##**Sección 2**
##**Grupo 23**
*   Rafael Santiago Bastos Russi - *202110792*
*   David Santiago Valderrama Herrera - *201910987*
*   Jesús Alejandro Dávila Pinchao - *202014263*




# **Caso de estudio**

La Organización de las Naciones Unidas (ONU) adopta, el 25 de septiembre del año 2015, la
Agenda 2030i para el desarrollo sostenible, cuyo fin es reducir la pobreza, garantizar acceso
a la salud y educación, buscar igualdad de género y oportunidades, disminuir el impacto
ambiental, entre otros. Esta agenda se basa en 17 objetivos de desarrollo sostenibleii (ODS)
y 169 metas (derivadas de los diferentes ODS).
Dentro del trabajo en conjunto de diferentes entes para alcanzar el cumplimiento de los
ODS, muchas entidades tienen como enfoque el seguimiento y la evaluación de las políticas
públicas y su impacto a nivel social. Este es el caso del Fondo de Poblaciones de las Naciones
Unidas (UNFPAiii) que, junto con entidades públicas y haciendo uso de diferentes
herramientas de participación ciudadana, busca identificar problemas y evaluar soluciones
actuales, relacionando la información con los diferentes ODS. En este contexto, uno de los
procesos que requiere de un mayor esfuerzo es la clasificación de la información textual
que es recopilada, ya que es una tarea que consume gran cantidad recursos y para la cual
se requiere un experto. Es por esto que, en los últimos años, UNFPA ha venido trabajando,
en conjunto con la Universidad de los Andes, en la implementación de diferentes estrategias
de clasificación de textos, que les permitan hacer un análisis automatizado de opiniones
que representan la voz de los habitantes locales sobre problemáticas de su entorno
particular.
Para apoyar a UNFPA en este proceso se ha planteado el primer proyecto del curso, cuyo
objetivo es desarrollar un modelo de clasificación, con técnicas de aprendizaje
automático, que permita relacionar de manera automática un texto según los ODS. Al
igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho
modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la
información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos
de planeación participativa para el desarrollo a nivel territorial.

#**Etapa 1**

## **1. Importación de librerias**

In [2]:
import pandas as pd
import numpy as np
import sys
import re, string, unicodedata

import inflect
import nltk
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion


In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
!python -m spacy download es_core_news_sm

2023-10-14 05:53:59.579351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


## **2. Perfilamiento y entendimiento de los datos**

### **2.1. Lectura de los datos**

In [5]:
data=pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos

### **2.2. Entendimiento de los datos**

In [25]:
data.shape

(3000, 2)

In [12]:
data.head()

,Textos_espanol,sdg
0,"Por ejemplo, el nÃºmero de consultas externas ...",3
1,"En 2007, el gobierno central financió directam...",3
2,"Claramente, hay muchos otros factores en juego...",3
3,"Por ejemplo, el estado australiano de Victoria...",3
4,"El consumo anual de alcohol se estima en 15,7 ...",3


In [13]:
data.dtypes

Textos_espanol    object
sdg                int64
dtype: object

In [9]:
data.describe()

,sdg
count,3000.000000
mean,4.000000
std,0.816633
min,3.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [14]:
pd.value_counts(data['sdg'])

3    1000
4    1000
5    1000
Name: sdg, dtype: int64

In [15]:
data.isnull().sum()

Textos_espanol    0
sdg               0
dtype: int64

In [21]:
data.duplicated(keep = False).sum()

0

## **3. Preparación de los datos**

### **3.1. Limpieza de los datos**

In [6]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
      new_word = word.lower()
      new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    stop_words = set(stopwords.words('english'))

    new_words = [word for word in words if word not in stop_words]
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

### **3.2. Tokenización**

In [15]:
data_t = data.copy()

In [16]:
nlp = spacy.load('es_core_news_sm')

In [17]:
texts = data_t['Textos_espanol']
tokens = []
for opinion in texts:
  tokens.append(nlp(opinion).text)



In [18]:
words = pd.Series(tokens, copy=False)

data_t['words_pro'] = words

In [19]:
data_t['words'] = data_t['words_pro'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
data_t.head()

,Textos_espanol,sdg,words_pro,words
0,"Por ejemplo, el nÃºmero de consultas externas ...",3,"Por ejemplo, el nÃºmero de consultas externas ...","[por, ejemplo, el, naomero, de, consultas, ext..."
1,"En 2007, el gobierno central financió directam...",3,"En 2007, el gobierno central financió directam...","[en, two thousand and seven, el, gobierno, cen..."
2,"Claramente, hay muchos otros factores en juego...",3,"Claramente, hay muchos otros factores en juego...","[claramente, hay, muchos, otros, factores, en,..."
3,"Por ejemplo, el estado australiano de Victoria...",3,"Por ejemplo, el estado australiano de Victoria...","[por, ejemplo, el, estado, australiano, de, vi..."
4,"El consumo anual de alcohol se estima en 15,7 ...",3,"El consumo anual de alcohol se estima en 15,7 ...","[el, consumo, anual, de, alcohol, se, estima, ..."


### **3.3. Normalización**

### **3.3.1. Stopwords**

In [20]:
stop_words = nlp.Defaults.stop_words

new_tokens = []
for tokens in data_t['words']:
  new_words = []
  for word in tokens:
    if word not in stop_words:
      new_words.append(word)
  new_tokens.append(new_words)

data_t['words'] = pd.Series(new_tokens, copy=False)


In [22]:
data_t.head()

,Textos_espanol,sdg,words_pro,words
0,"Por ejemplo, el nÃºmero de consultas externas ...",3,"Por ejemplo, el nÃºmero de consultas externas ...","[ejemplo, naomero, consultas, externas, especi..."
1,"En 2007, el gobierno central financió directam...",3,"En 2007, el gobierno central financió directam...","[two thousand and seven, gobierno, central, fi..."
2,"Claramente, hay muchos otros factores en juego...",3,"Claramente, hay muchos otros factores en juego...","[claramente, factores, juego, particular, vari..."
3,"Por ejemplo, el estado australiano de Victoria...",3,"Por ejemplo, el estado australiano de Victoria...","[ejemplo, australiano, victoria, agencia, prom..."
4,"El consumo anual de alcohol se estima en 15,7 ...",3,"El consumo anual de alcohol se estima en 15,7 ...","[consumo, anual, alcohol, estima, 157, litros,..."
